In [ ]:
import warnings

warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
eanCodeLists = []
for page in range(2, 9):
    url = f"https://www.bigbasket.com/product/get-products/?slug=dals-pulses&page={page}&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc"
    print(url)
    # headers
    headers = {
        'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
    }

    json = requests.get(url).json()
    prod_list = json['tab_info']['product_map']['all']['prods']
    print(len(prod_list))
    for prod in prod_list:
        if len(prod['all_prods']) > 0:
            for pro in prod['all_prods']:
                eanCodeLists.append(pro['sku'])
        else:
            print('No inner product')
            eanCodeLists.append(prod['sku'])
        print('\n\n')
            
print(len(eanCodeLists))

In [ ]:
ProductNameList = []
BrandNameList = []
ProductQtyList = []
ProductPriceList = []
ProductDescList = []

for items in eanCodeLists:
    url = "https://www.bigbasket.com/pd/" + str(items)
    urlopen = requests.get(url).text
    data = BeautifulSoup(urlopen, 'lxml')
    ProductInfo = data.find("h1", {"class": "GrE04"}).text.strip()
    productName = ProductInfo.split(' ',1)[1].split(',')[0]
    ProductNameList.append(productName)
    brandName = data.find("a", {"class": "_2zLWN"}).text.strip()
    BrandNameList.append(brandName)
    prodQty = ProductInfo.split(' ',1)[1].split(',')[1]
    ProductQtyList.append(prodQty)
    prodPrice = data.find("td", {"data-qa": "productPrice"}).text.strip()
    ProductPriceList.append(prodPrice)
    prodDesc = data.find("div", {"class": "_26MFu"}).text.strip()
    ProductDescList.append(prodDesc)

In [ ]:
table = {'ProductName': ProductNameList, 'BrandName': BrandNameList, 'Product Quantity': ProductQtyList, 'Product Price': ProductPriceList, 'Product Description': ProductDescList}
df = pd.DataFrame(table)
df.head()